* 익스 예시

In [11]:
import os, re 
import numpy as np
import tensorflow as tf

# 파일을 읽기모드로 열고
# 라인 단위로 끊어서 list 형태로 읽어옵니다.
file_path = os.getenv('HOME') + '/aiffel/lyricist/data/shakespeare.txt'
with open(file_path, "r") as f:
    raw_corpus = f.read().splitlines()

# 앞에서부터 10라인만 화면에 출력해 볼까요?
print(raw_corpus[:9],'\n')
print("데이터 크기:", len(raw_corpus))

['First Citizen:', 'Before we proceed any further, hear me speak.', '', 'All:', 'Speak, speak.', '', 'First Citizen:', 'You are all resolved rather to die than to famish?', ''] 

데이터 크기: 40000


## 학습 요구사항
1. 데이터 정제
- 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기 를 권합니다.   
leng = len(sentence.split())로 추가 리턴해서 받아서 했습니다
2. 평가 데이터셋 분리
- tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!      

num_words=13000, test_size=0.2   
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, test_size=0.2,shuffle=True, 
                                                          random_state=66)

3. 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)   
Epoch 8/8   
502/502 [==============================] - 500s 996ms/step - loss: 1.1447 - val_loss: 2.1226   
달성!

## 과제 시작 - 데이터 읽어오기

In [2]:
import glob
import os # Operating System 불러오기 운영체제처럼 파일 입출력 처리가 가능
import re # 정규계산식 불러오기 preprocess_sentence에서 sub사용 - 파일 전처리(공백제거, 소문자화, 특수문자제거)
import numpy as np 
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:#위 경로로 read하기
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 [' There must be some kind of way outta here', 'Said the joker to the thief', "There's too much confusion"]


In [14]:
# 입력된 문장을
# 1. 소문자로 바꾸고=lower, strip = 양쪽 공백을 지웁니다
# 2. 특수문자([?.!,¿]) 양쪽에 공백을 넣고(r" \1 ") sentence가 \1자리에 들어감 - re.sub（정규 표현식, 바뀔 문자 , 치환 문자)
# 3. 여러개의 공백은 하나의 공백으로 바꿉니다 - r 문자는 raw string으로 백슬래시 문자를 해석하지 않고 남겨두기 때문에 정규표현식과 같은 곳에 유용하다.
# 4. a-zA-Z?.!,¿가 아닌 모든 문자를 하나의 공백으로 바꿉니다
# 5. 다시 양쪽 공백을 지웁니다
# 6. 문장 시작에는 <start>, 끝에는 <end>를 추가합니다
# 이 순서로 처리해주면 문제가 되는 상황을 방지할 수 있겠네요!
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = re.sub(r"([?.!,¿])", " ", sentence) #한번더 해서 특수문자 제거 ###
    sentence = '<start> ' + sentence + ' <end>' # 6
    leng = len(sentence.split()) # 토큰 갯수 측정
    #print(leng) 시험용
    return sentence, leng #반환에 추가

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample    what the    sentence ."))

('<start> this is sample what the sentence   <end>', 8)


In [15]:
# 여기에 정제된 문장을 모을겁니다
corpus = []
preprocessed_sentence = [] #안해주면 리턴값 받을 때 뭘받을지 구분을 못하더군요
for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
       
    # 정제를 하고 담아주세요
    preprocessed_sentence, leng = preprocess_sentence(sentence)
    if leng > 15: #토큰 15개 이상 어펜드 안함
        continue
    corpus.append(preprocessed_sentence)
        
# 정제된 결과를 10개만 확인해보죠
corpus[:10]

['<start> there must be some kind of way outta here <end>',
 '<start> said the joker to the thief <end>',
 '<start> there s too much confusion <end>',
 '<start> i can t get no relief business men   they drink my wine <end>',
 '<start> plowman dig my earth <end>',
 '<start> none were level on the mind <end>',
 '<start> nobody up at his word <end>',
 '<start> hey   hey no reason to get excited <end>',
 '<start> the thief he kindly spoke <end>',
 '<start> there are many here among us <end>']

In [31]:
# 토큰화 할 때 텐서플로우의 Tokenizer와 pad_sequences를 사용합니다
# 더 잘 알기 위해 아래 문서들을 참고하면 좋습니다
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/text/Tokenizer
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/sequence/pad_sequences

def tokenize(corpus):
    # 7000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 7000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
       # document_count=15,
        num_words=13000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    word_dic = tokenizer.word_index
    #print(type(word_dic)) 안받아져서 확인결과 dict이였다
    for x,i in word_dic.items(): #인덱스가 뒤에나오더라구요 ㅠ
        print(i,x )
        if i >= 10:
            break
    
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 문장 앞에 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용합니다
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, #maxlen=15, 
                                                           padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

1 <unk>
2 <start>
3 <end>
4 i
5 the
6 you
7 and
8 a
9 to
10 it
11 me
[[  2  58 269 ...   0   0   0]
 [  2 107   5 ...   0   0   0]
 [  2  58  16 ...   0   0   0]
 ...
 [  2  73  44 ...   3   0   0]
 [  2  46 697 ...   0   0   0]
 [  2  12 637 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7fe2d74a5a10>


In [17]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

enc_train, enc_val, dec_train, dec_val = train_test_split(src_input,tgt_input, test_size=0.2,shuffle=True, 
                                                          random_state=66)
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (128302, 14)
Target Train: (128302, 14)


In [18]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out) 
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 625
hidden_size = 2000
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [19]:
# optimizer와 loss등은 차차 배웁니다
# 혹시 미리 알고 싶다면 아래 문서를 참고하세요
# https://www.tensorflow.org/api_docs/python/tf/keras/optimizers
# https://www.tensorflow.org/api_docs/python/tf/keras/losses
# 양이 상당히 많은 편이니 지금 보는 것은 추천하지 않습니다
optimizer = tf.keras.optimizers.Adam()

loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train,dec_train,epochs=8,validation_data=(enc_val, dec_val),batch_size=256,verbose=1)
# 하이퍼파라메터 시험중 히든 수치가 높을때 로스값이 오버피팅되어서 과제값은 10이였지만 8로 가공

Epoch 1/8
502/502 [==============================] - 518s 994ms/step - loss: 3.8046 - val_loss: 2.8846
Epoch 2/8
502/502 [==============================] - 501s 997ms/step - loss: 2.7587 - val_loss: 2.6223
Epoch 3/8
502/502 [==============================] - 500s 996ms/step - loss: 2.4096 - val_loss: 2.4368
Epoch 4/8
502/502 [==============================] - 500s 997ms/step - loss: 2.0695 - val_loss: 2.3022
Epoch 5/8
502/502 [==============================] - 501s 997ms/step - loss: 1.7577 - val_loss: 2.2091
Epoch 6/8
502/502 [==============================] - 500s 996ms/step - loss: 1.4949 - val_loss: 2.1517
Epoch 7/8
502/502 [==============================] - 500s 996ms/step - loss: 1.2887 - val_loss: 2.1229
Epoch 8/8
502/502 [==============================] - 500s 996ms/step - loss: 1.1447 - val_loss: 2.1226


---
#### 성공 박제
Epoch 1/8
502/502 [==============================] - 518s 994ms/step - loss: 3.8046 - val_loss: 2.8846   
Epoch 2/8
502/502 [==============================] - 501s 997ms/step - loss: 2.7587 - val_loss: 2.6223   
Epoch 3/8
502/502 [==============================] - 500s 996ms/step - loss: 2.4096 - val_loss: 2.4368   
Epoch 4/8
502/502 [==============================] - 500s 997ms/step - loss: 2.0695 - val_loss: 2.3022   
Epoch 5/8
502/502 [==============================] - 501s 997ms/step - loss: 1.7577 - val_loss: 2.2091   
Epoch 6/8
502/502 [==============================] - 500s 996ms/step - loss: 1.4949 - val_loss: 2.1517   
Epoch 7/8
502/502 [==============================] - 500s 996ms/step - loss: 1.2887 - val_loss: 2.1229   
Epoch 8/8
502/502 [==============================] - 500s 996ms/step - loss: 1.1447 - val_loss: 2.1226   
- 오버피팅이 올려다 마는 곳에서 반복이 끊겼습니다 실패의 파라미터가 도움이 되었습니다 2.1226!
---

# 하이퍼파라메터 실패의 기록(전처리 개선없는)

기본값 embedding_size = 256 hidden_size = 1024를 했을 때   
Epoch 1/10
550/550 [==============================] - 194s 344ms/step - loss: 4.2613 - val_loss: 3.3519   
Epoch 7/10
550/550 [==============================] - 189s 343ms/step - loss: 2.5699 - val_loss: 2.7718   
Epoch 8/10
550/550 [==============================] - ETA: 0s - loss: 2.4716
* 으 30분 가까이 돌렸는데 lms로그아웃으로 끊어졌습니다 ㅠㅠ    
* 돌린 시간이 아까우니 남깁니다


embedding_size = 25 hidden_size = 2048를 했을 때

Epoch 10/10
550/550 [==============================] - 512s 932ms/step - loss: 1.7950 - val_loss: 2.5254
* 나아진것이 보이나 목표치인 2.2에는 도달못함 히든사이즈가 높으면   
    감당못할정도로 시간이 듬

embedding_size = 25
hidden_size = 756

Epoch 10/10
550/550 [==============================] - 131s 238ms/step - loss: 2.7618 - val_loss: 2.9142
* 히든좀 줄였더니 빨라졌는데 로스 더내려감 으어어

embedding_size = 625
hidden_size = 756

Epoch 10/10
550/550 [==============================] - 155s 281ms/step - loss: 2.3668 - val_loss: 2.6614
* 엠베딩 상승도 올라가는 것처럼 보임

embedding_size = 625
hidden_size = 1300

Epoch 10/10
550/550 [==============================] - 293s 532ms/step - loss: 1.3425 - val_loss: 2.3373

embedding_size = 1225
hidden_size = 1300

Epoch 9/10
550/550 [==============================] - 336s 611ms/step - loss: 1.4462 - val_loss: 2.3212   
Epoch 10/10
550/550 [==============================] - 337s 612ms/step - loss: 1.2974 - val_loss: 2.3233
* 일정 수준이상의 임베딩 효과 미묘함. 2.2 너무 멀어요

embedding_size = 625
hidden_size = 2500   
Epoch 8/10
550/550 [==============================] - 763s 1s/step - loss: 1.2173 - val_loss: 2.2563   
Epoch 9/10
550/550 [==============================] - 761s 1s/step - loss: 1.0600 - val_loss: 2.2686   
Epoch 10/10
550/550 [==============================] - 762s 1s/step - loss: 0.9620 - val_loss: 2.2943   

* 역시 물량이 최곱니다 으어어 시간을 때려박아 만들었습니다.    
    하지만 보시는대로 8이후 과적합이 생기는 군요
    어마어마한 시간을 들였지만 간신히 목표 달성 언저리네요


---
### 대망의 학습모델로 텍스트 만들기

In [32]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

---
### 문장 만들어보기

In [37]:
generate_text(model, tokenizer, init_sentence="<start> i", max_len=30)

'<start> i m the one yeah oh eh oh oh oh oh eh oh <end> '

In [36]:
generate_text(model, tokenizer, init_sentence="<start> my", max_len=30)

'<start> my love for you is the real kind <end> '

In [38]:
generate_text(model, tokenizer, init_sentence="<start> god", max_len=30)

'<start> god knows we can get all the way from here to there <end> '

In [42]:
generate_text(model, tokenizer, init_sentence="<start> river", max_len=30)

'<start> river of no return <end> '

In [43]:
generate_text(model, tokenizer, init_sentence="<start> like", max_len=30)

'<start> like a rolling stone <end> '

---
## 진행고통록
과제를 진행하는중    
- Source Train: (140599, 346)가 나온다 토큰화 개수는 어디서 봐서 줄일까      
- padsequences에서 maxlen=15를 줘봤더니 일단 와이축은 맞춰졌다    
- tokenizer에서  document_count=15했지만 변함없다. 뭐가 틀렸지   
- 일단 안되는 것을 두고 뒤로 가보기로 했습니다   
- 뒤에서 하이퍼 파라메터만으로는 2.2의 고지가 멀다는걸 엄청난 경험과 시간뒤 ㅠ 배웠습니다   
- maxlen은 짤라버리는거라 토큰을 손상시키니 도로 빼고
- 전처리과정과 정리과정에서 수를 드디어 좀 줄였습니다(토큰 15개 초과시 제거) (128302, 14)  
- 하이퍼파라메터 시험중 히든 수치가 높을때 로스값이 오버피팅되어서 과제값은 10이내 였지만 8로 가공
---

---
## 회고
- 이야 학습시간이 어마어마해서 돌리고 돌리고 또 돌렸습니다. 해냈습니다!
- 다른 대단한 분들과는 다르게 전 뭔가 고찰이 안되고 그냥 우왕좌왕 실패록인거 같습니다만 그래도 기쁩니다
